In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
import random ,json
import numpy as np


import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, RandomSampler, Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from torch.optim import AdamW
from tqdm import tqdm

## Setting Basic Parameters


In [ ]:
class Config:
    batch_size = 4
    epochs = 6
    lr = 1e-5
    seed = 123


    # Can only have one True
    # only_use_standard_linear_layer=True, use linear layer
    only_use_standard_linear_layer = True
    # Setting the linear layer number
    linear_layer_num = 1
    linear_layer = [512, 512, 512, 512, 512, 512, 512, 512]


    # if only_use_standard_linear_layer is False, only_use_dropout is True, it means just use dropout.
    only_use_dropout = False
    dropout_prob = [0.05, 0, 0.05]


    # if only_use_standard_linear_layer is False, only_use_residual = True, just use resdiual
    only_use_residual = False
    # if only_use_standard_linear_layer is False, only_use_residual_and_dropout = True, use residual and dropout together.
    only_use_residual_and_dropout = False



    assert sum([1 if only_use_standard_linear_layer else 0,
                1 if only_use_dropout else 0,
                1 if only_use_residual else 0,
                1 if only_use_residual_and_dropout else 0]) == 1



    # if lstm layer is 0, then not using the lstm layer. Setting the lstm layer based on layers number required
    lstm_layer_num = 0
    bi_lstm=True


    # Internet resource; download from Internet
    # model_name = "microsoft/deberta-v3-base"

    model_name = "microsoft/deberta-base"
    #model_name = "deberta-base"

    hidden_size=768
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_data_name = "conll2003"

    @classmethod
    def describe(cls):
        parm = {"train_data_name": cls.train_data_name,
                "encoder_name": cls.model_name,
                "batch_size": cls.batch_size,
                "epochs": cls.epochs,
                "lr": cls.lr,
                "seed": cls.seed,
                "bi_lstm": cls.bi_lstm,
                "lstm_layer_num": cls.lstm_layer_num,
                "linear_layer": cls.linear_layer,
                "linear_layer_num":cls.linear_layer_num,
                "dropout_prob": cls.dropout_prob,
                "only_use_standard_linear_layer": cls.only_use_standard_linear_layer,
                "only_use_dropout": cls.only_use_dropout,
                "only_use_residual": cls.only_use_residual,
                "only_use_residual_and_dropout": cls.only_use_residual_and_dropout}
        return json.dumps(parm , ensure_ascii=False, indent=2)

random.seed(Config.seed)
np.random.seed(Config.seed)
torch.manual_seed(Config.seed)
torch.cuda.manual_seed_all(Config.seed)



## given configuration result

In [ ]:
print(Config.describe())

{
  "train_data_name": "conll2003",
  "encoder_name": "microsoft/deberta-base",
  "batch_size": 4,
  "epochs": 6,
  "lr": 1e-05,
  "seed": 123,
  "bi_lstm": true,
  "lstm_layer_num": 0,
  "linear_layer": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "linear_layer_num": 1,
  "dropout_prob": [
    0.05,
    0,
    0.05
  ],
  "only_use_standard_linear_layer": true,
  "only_use_dropout": false,
  "only_use_residual": false,
  "only_use_residual_and_dropout": false
}


## Import dataset

In [ ]:
def read_conll2003(file_path):
    data = []
    sample = []
    for idx, line in enumerate(open(file_path)):
        if idx == 0:
            continue
        line = line.strip()
        if line == "":
            if len(sample) != 0:
                data.append(sample)
            sample = []
        else:
            line = line.split()
            assert len(line) == 4
            sample.append([line[0], line[-1]])
    if len(sample) != 0:
        data.append(sample)
    data = [{"word": [i[0] for i in sample], "tag": [i[1] for i in sample]} for sample in data]
    return pd.DataFrame(data)

# Setting the chosen dataset, conll2003 or ner_datasetreference.
if Config.train_data_name == "conll2003":
    train_path = os.path.join(Config.train_data_name, 'train.txt')
    dev_path = os.path.join(Config.train_data_name, 'valid.txt')
    test_path = os.path.join(Config.train_data_name, 'test.txt')
    train_df = read_conll2003(train_path)
    valid_df = read_conll2003(dev_path)
    test_df = read_conll2003(test_path)
    print(train_df.shape, valid_df.shape, test_df.shape)
elif Config.train_data_name == "ner_datasetreference":
    df = pd.read_csv("ner_datasetreference.csv", encoding='iso-8859-1')
    data = []
    word, tag = [], []
    for i,j,k in zip(df['Sentence #'], df['Word'], df['Tag']):
        if not pd.isnull(i):
            assert i.startswith('Sentence')
            if len(word) > 0:
                data.append({"word":word, "tag":tag})
            word, tag = [], []
        if isinstance(j, str) and isinstance(k, str):
            # remove 'art', 'eve', 'nat' label for better macro results
            if any( t in k for t in ['art', 'eve', 'nat']):
                continue
            word.append(j)
            tag.append(k)
    if len(word) > 0:
        data.append({"word":word, "tag":tag})
        word, tag = [], []
    print(data[0], data[-1])
    df = pd.DataFrame(data)
    train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
    valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    print(df.shape, train_df.shape, valid_df.shape, test_df.shape)

(14986, 2) (3465, 2) (3683, 2)


In [ ]:
train_df

,word,tag
0,"[EU, rejects, German, call, to, boycott, Briti...","[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]"
1,"[Peter, Blackburn]","[B-PER, I-PER]"
2,"[BRUSSELS, 1996-08-22]","[B-LOC, O]"
3,"[The, European, Commission, said, on, Thursday...","[O, B-ORG, I-ORG, O, O, O, O, O, O, B-MISC, O,..."
4,"[Germany, 's, representative, to, the, Europea...","[B-LOC, O, O, O, O, B-ORG, I-ORG, O, O, O, B-P..."
...,...,...
14981,"[on, Friday, :]","[O, O, O]"
14982,"[Division, two]","[O, O]"
14983,"[Plymouth, 2, Preston, 1]","[B-ORG, O, B-ORG, O]"
14984,"[Division, three]","[O, O]"


In [ ]:
test_df

,word,tag
0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, ...","[O, O, B-LOC, O, O, O, O, B-PER, O, O, O, O]"
1,"[Nadim, Ladki]","[B-PER, I-PER]"
2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]","[B-LOC, O, B-LOC, I-LOC, I-LOC, O]"
3,"[Japan, began, the, defence, of, their, Asian,...","[B-LOC, O, O, O, O, O, B-MISC, I-MISC, O, O, O..."
4,"[But, China, saw, their, luck, desert, them, i...","[O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O,..."
...,...,...
3678,"[That, is, why, this, is, so, emotional, a, ni...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER,..."
3679,"["", It, was, the, joy, that, we, all, had, ove...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3680,"[Charlton, managed, Ireland, for, 93, matches,...","[B-PER, O, B-LOC, O, O, O, O, O, O, O, O, O, O..."
3681,"[He, guided, Ireland, to, two, successive, Wor...","[O, O, B-LOC, O, O, O, B-MISC, I-MISC, O, O, O..."


In [ ]:
valid_df

,word,tag
0,"[CRICKET, -, LEICESTERSHIRE, TAKE, OVER, AT, T...","[O, O, B-ORG, O, O, O, O, O, O, O, O]"
1,"[LONDON, 1996-08-30]","[B-LOC, O]"
2,"[West, Indian, all-rounder, Phil, Simmons, too...","[B-MISC, I-MISC, O, B-PER, I-PER, O, O, O, O, ..."
3,"[Their, stay, on, top, ,, though, ,, may, be, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG,..."
4,"[After, bowling, Somerset, out, for, 83, on, t...","[O, O, B-ORG, O, O, O, O, O, O, O, O, B-LOC, I..."
...,...,...
3460,"[But, the, prices, may, move, in, a, close, ra...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3461,"[Brokers, said, blue, chips, like, IDLC, ,, Ba...","[O, O, O, O, O, B-ORG, O, B-ORG, I-ORG, O, B-O..."
3462,"[They, said, there, was, still, demand, for, b...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3463,"[The, DSE, all, share, price, index, closed, 2...","[O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O,..."


In [ ]:
def collect_label(df_list):
    ret = set()
    for df in df_list:
        for labels in df['tag']:
            for l in labels:
                if l == "O":
                    continue
                assert l.startswith("B-") or l.startswith("I-")
                ret.add(l[2:])
    return sorted(list(ret))

ner_category = collect_label([train_df, valid_df, test_df])
label_list = []
for l in ner_category:
    label_list.append("B-" + l)
    label_list.append("I-" + l)
label_list = ['O'] + label_list
label2id = dict([(v, idx) for idx, v in enumerate(label_list)])
id2label = dict([(idx, v) for idx, v in enumerate(label_list)])
print(f"ner category {ner_category} .\n\nlabel list {label_list} .\n\nlabel2id {label2id} .\n\nid2label {id2label}\n\n")
label_list = label_list

ner category ['LOC', 'MISC', 'ORG', 'PER'] .

label list ['O', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER'] .

label2id {'O': 0, 'B-LOC': 1, 'I-LOC': 2, 'B-MISC': 3, 'I-MISC': 4, 'B-ORG': 5, 'I-ORG': 6, 'B-PER': 7, 'I-PER': 8} .

id2label {0: 'O', 1: 'B-LOC', 2: 'I-LOC', 3: 'B-MISC', 4: 'I-MISC', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-PER', 8: 'I-PER'}




## Import Reberta Model

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(Config.model_name, add_prefix_space=True)
print(tokenizer.is_fast)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

True


## tokenize and build Label

In [ ]:
def align(tag, word_ids):
    aligned_tag = []
    i = 0
    while i < len(word_ids):
        if word_ids[i] is None:
            aligned_tag.append(None)
            i += 1
        elif tag[word_ids[i]] == "O":
            aligned_tag.append(tag[word_ids[i]])
            i += 1
        elif tag[word_ids[i]].startswith("B-"):
            n = 0
            while (i+n) < len(word_ids) and word_ids[i]  == word_ids[i+n]:
                n += 1
            aligned_tag.append(tag[word_ids[i]])
            if n > 1:
                aligned_tag.extend(["I-" + tag[word_ids[i]][2:] ] * (n-1))
            i = i + n
        else:
            aligned_tag.append(tag[word_ids[i]])
            i += 1
    return aligned_tag


In [ ]:
#words = train_df.iloc[2]["word"]
#tag = train_df.iloc[2]["label"]
words = ['I', '1996-08-22', '1996-08-22', 'I']
tag = ["O", "B-LOC", "B-ORG", "O"]
print(words, tag)
s = tokenizer(words, truncation=True, is_split_into_words=True)
word_ids = s.word_ids()
# align tokens and words
tokens = tokenizer.convert_ids_to_tokens(s['input_ids'])
tags = align(tag, s.word_ids())
print(pd.DataFrame(list(zip(tokens, tags, word_ids)), columns=['tokens', 'tags', 'word-index']))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['I', '1996-08-22', '1996-08-22', 'I'] ['O', 'B-LOC', 'B-ORG', 'O']
   tokens   tags  word-index
0   [CLS]   None         NaN
1      ĠI      O         0.0
2   Ġ1996  B-LOC         1.0
3       -  I-LOC         1.0
4      08  I-LOC         1.0
5       -  I-LOC         1.0
6      22  I-LOC         1.0
7   Ġ1996  B-ORG         2.0
8       -  I-ORG         2.0
9      08  I-ORG         2.0
10      -  I-ORG         2.0
11     22  I-ORG         2.0
12     ĠI      O         3.0
13  [SEP]   None         NaN


In [ ]:
def preprocess(x):
    word = x['word']
    r = tokenizer(word, truncation=True, is_split_into_words=True)
    word_ids = r.word_ids()
    tokens = tokenizer.convert_ids_to_tokens(r['input_ids'])
    align_label = align(x['tag'], word_ids)
    return tokens, align_label, r['input_ids'], [label2id[i] if i is not None else -100  for i in align_label], word_ids


In [ ]:
train_df[['token', 'label', 'id', 'label_id', 'word_ids']] = train_df.apply(lambda x: pd.Series(preprocess(x)), axis=1)
valid_df[['token', 'label', 'id', 'label_id', 'word_ids']] = valid_df.apply(lambda x: pd.Series(preprocess(x)), axis=1)
test_df[['token', 'label', 'id', 'label_id', 'word_ids']] = test_df.apply(lambda x: pd.Series(preprocess(x)), axis=1)

In [ ]:
test_df

,word,tag,token,label,id,label_id,word_ids
0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, ...","[O, O, B-LOC, O, O, O, O, B-PER, O, O, O, O]","[[CLS], ĠSO, CC, ER, Ġ-, ĠJ, AP, AN, ĠGET, ĠL,...","[None, O, O, O, O, B-LOC, I-LOC, I-LOC, O, O, ...","[1, 13910, 3376, 2076, 111, 344, 591, 1889, 77...","[-100, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, ...","[None, 0, 0, 0, 1, 2, 2, 2, 3, 4, 4, 4, 5, 6, ..."
1,"[Nadim, Ladki]","[B-PER, I-PER]","[[CLS], ĠNad, im, ĠLad, ki, [SEP]]","[None, B-PER, I-PER, I-PER, I-PER, None]","[1, 7157, 757, 17804, 3144, 2]","[-100, 7, 8, 8, 8, -100]","[None, 0, 0, 1, 1, None]"
2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]","[B-LOC, O, B-LOC, I-LOC, I-LOC, O]","[[CLS], ĠAL, -, AIN, Ġ,, ĠUnited, ĠArab, ĠEmir...","[None, B-LOC, I-LOC, I-LOC, O, B-LOC, I-LOC, I...","[1, 6019, 12, 33178, 2156, 315, 4681, 8313, 80...","[-100, 1, 2, 2, 0, 1, 2, 2, 0, 0, 0, 0, 0, -100]","[None, 0, 0, 0, 1, 2, 3, 4, 5, 5, 5, 5, 5, None]"
3,"[Japan, began, the, defence, of, their, Asian,...","[B-LOC, O, O, O, O, O, B-MISC, I-MISC, O, O, O...","[[CLS], ĠJapan, Ġbegan, Ġthe, Ġdefence, Ġof, Ġ...","[None, B-LOC, O, O, O, O, O, B-MISC, I-MISC, O...","[1, 1429, 880, 5, 2994, 9, 49, 3102, 968, 1270...","[-100, 1, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1..."
4,"[But, China, saw, their, luck, desert, them, i...","[O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O,...","[[CLS], ĠBut, ĠChina, Ġsaw, Ġtheir, Ġluck, Ġde...","[None, O, B-LOC, O, O, O, O, O, O, O, O, O, O,...","[1, 125, 436, 794, 49, 6620, 10348, 106, 11, 5...","[-100, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1..."
...,...,...,...,...,...,...,...
3678,"[That, is, why, this, is, so, emotional, a, ni...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER,...","[[CLS], ĠThat, Ġis, Ġwhy, Ġthis, Ġis, Ġso, Ġem...","[None, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1, 280, 16, 596, 42, 16, 98, 3722, 10, 363, 1...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1..."
3679,"["", It, was, the, joy, that, we, all, had, ove...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[CLS], Ġ"", ĠIt, Ġwas, Ġthe, Ġjoy, Ġthat, Ġwe,...","[None, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1, 22, 85, 21, 5, 5823, 14, 52, 70, 56, 81, 5...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1..."
3680,"[Charlton, managed, Ireland, for, 93, matches,...","[B-PER, O, B-LOC, O, O, O, O, O, O, O, O, O, O...","[[CLS], ĠCharl, ton, Ġmanaged, ĠIreland, Ġfor,...","[None, B-PER, I-PER, O, B-LOC, O, O, O, O, O, ...","[1, 15959, 1054, 2312, 2487, 13, 8060, 2856, 2...","[-100, 7, 8, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[None, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11..."
3681,"[He, guided, Ireland, to, two, successive, Wor...","[O, O, B-LOC, O, O, O, B-MISC, I-MISC, O, O, O...","[[CLS], ĠHe, Ġguided, ĠIreland, Ġto, Ġtwo, Ġsu...","[None, O, O, B-LOC, O, O, O, B-MISC, I-MISC, O...","[1, 91, 10346, 2487, 7, 80, 12565, 623, 968, 6...","[-100, 0, 0, 1, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1..."


## Building Dataloader

In [ ]:

class NerDataset(Dataset):
    def __init__(self, df, device):
        self.data = df.to_dict(orient='records')
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.data[item]

    def collate_to_max_length(self, batch):
        max_seq_length = max([len(s['id']) for s in batch])
        batch = sorted(batch, key=lambda x: -len(x['id']))
        seq_length = torch.tensor([len(x['id']) for x in batch])
        input_ids = torch.tensor([x["id"] + [0] * (max_seq_length - len(x['id'])) for x in batch]).to(self.device)
        labels = torch.tensor([x["label_id"] + [-100] * (max_seq_length - len(x['label_id'])) for x in batch]).to(self.device)
        return {"id": input_ids, "label_id": labels, 'seq_length':seq_length, "sample":batch}


dataset_train = NerDataset(train_df, Config.device)

train_dataloader = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=Config.batch_size,
                              drop_last=False,
                              collate_fn=dataset_train.collate_to_max_length)



dataset_valid = NerDataset(valid_df, Config.device)

valid_dataloader = DataLoader(dataset_valid,
                              sampler=RandomSampler(dataset_valid),
                              batch_size=Config.batch_size,
                              drop_last=False,
                              collate_fn=dataset_valid.collate_to_max_length)


dataset_test = NerDataset(test_df, Config.device)

test_dataloader = DataLoader(dataset_test,
                              sampler=RandomSampler(dataset_test),
                              batch_size=Config.batch_size,
                              drop_last=False,
                              collate_fn=dataset_test.collate_to_max_length)

## Building Custom loss functions

In [ ]:

class L1_Loss:
    def __init__(self):
        self.l1_loss = nn.L1Loss()
    def loss(self, target, logit, label_num):

        target = target.view(-1)
        logit = logit.view(-1, label_num)
        mask = target.ne(-100).to(logit.device)
        logit = torch.masked_select(logit, mask.unsqueeze(-1).expand_as(logit)).reshape(-1, label_num)
        target = torch.masked_select(target, mask)

        target = F.one_hot(target, num_classes=label_num)
        return self.l1_loss(logit, target.float())


class L2_Loss:
    def __init__(self):
        self.mse_loss = nn.MSELoss()
    def loss(self, target, logit,label_num):
        target = target.view(-1)
        logit = logit.view(-1, label_num)
        mask = target.ne(-100).to(logit.device)
        logit = torch.masked_select(logit, mask.unsqueeze(-1).expand_as(logit)).reshape(-1, label_num)
        target = torch.masked_select(target, mask)

        target = F.one_hot(target, num_classes=label_num)
        loss = self.mse_loss(logit, target.float())
        return loss

class CE_Loss:
    def __init__(self):
        self.ce_loss = nn.CrossEntropyLoss(ignore_index=-100, reduce='mean')
    def loss(self, target, logit, label_num):
        return self.ce_loss(logit.reshape(-1, label_num), target.reshape(-1) )

class KLDivergenceLoss:
    def __init__(self):
        pass

    def loss(self, target, logit, label_num):
        target = target.view(-1)
        logit = logit.view(-1, label_num)

        mask = target.ne(-100).to(logit.device)
        logit = torch.masked_select(logit, mask.unsqueeze(-1).expand_as(logit)).reshape(-1, label_num)
        target = torch.masked_select(target, mask)

        probs = F.softmax(logit, dim=-1)

        # One-hot encode the targets to get true probabilities
        true_probs = F.one_hot(target, num_classes=label_num).float()

        mask_true_probs = true_probs > 0

        # Calculate g function for non-zero elements using the mask
        kl_values = torch.zeros_like(probs)
        kl_values[mask_true_probs] = true_probs[mask_true_probs] * torch.log(true_probs[mask_true_probs]/probs[mask_true_probs])

        # Sum over all classes and average over the batch size
        loss = kl_values.sum(dim=-1).mean()

        return loss

# DLITE Loss function
class DLITELoss:
    def __init__(self):
        super(DLITELoss, self).__init__()

    def loss(self, targets, logits, label_num, epsilon=1e-10):
        targets = targets.view(-1)
        logits = logits.view(-1, label_num)

        mask = targets.ne(-100).to(logits.device)
        logits = torch.masked_select(logits, mask.unsqueeze(-1).expand_as(logits)).reshape(-1, label_num)
        targets = torch.masked_select(targets, mask)

        # Convert logits to probabilities using softmax
        probs = F.softmax(logits, dim=-1)

        # One-hot encode the targets to get true probabilities
        true_probs = F.one_hot(targets, num_classes=probs.size(-1)).float()

        # Define the g function
        g_values = torch.abs(probs * (1 - torch.log(probs + epsilon)) - true_probs * (1 - torch.log(true_probs + epsilon)))

        # Define the delta_h function
        delta_h_values = torch.abs(probs**2 * (1 - 2 * torch.log(probs + epsilon)) - true_probs**2 * (1 - 2 * torch.log(true_probs + epsilon))) / (2 * (probs + true_probs))

        # Compute DLITE loss for each class
        dl_values = g_values - delta_h_values

        # Sum over all classes and average over batch size
        loss = dl_values.sum(dim=-1).mean()

        return loss



## Adding Custom Layer

In [ ]:

class LSTMEncoder(nn.Module):
    """lstm encoder
    """
    def __init__(self, config, hidden_size):
        super(LSTMEncoder, self).__init__()
        self.lstm = torch.nn.LSTM(hidden_size, hidden_size,
                                  num_layers=config.lstm_layer_num, bidirectional=config.bi_lstm,
                                  batch_first=True)

    def forward(self, hidden_state, seq_length):
        sequence_output = pack_padded_sequence(hidden_state, seq_length, batch_first=True)
        sequence_output, (h_n, c_n) = self.lstm(sequence_output)
        sequence_output, _ = pad_packed_sequence(sequence_output, batch_first=True)
        return sequence_output







class LinearResidualLayer(nn.Module):
    def __init__(self, config, hidden_size, output_dim):
        super(LinearResidualLayer, self).__init__()
        self.config = config
        self.linear_layer1 = nn.Linear(in_features=hidden_size, out_features=output_dim)
        self.linear_layer2 = nn.Linear(in_features=output_dim, out_features=output_dim)
        self.linear_layer3 = nn.Linear(in_features=output_dim, out_features=output_dim)
        self.act_func = nn.ReLU()
        if not self.config.only_use_residual:
            self.dropout1 = nn.Dropout(config.dropout_prob[0])
        self.ln_1 = nn.LayerNorm(output_dim)
        if not self.config.only_use_residual:
            self.dropout2 = nn.Dropout(config.dropout_prob[1])



    def forward(self, x):
        # x = self.act_func(self.linear_layer1(x))
        # x = self.ln_1(x)

        # y = self.dropout1(x) + self.dropout1(self.act_func(self.linear_layer2(x) )  )
        # y = self.ln_2(y)
        # z = self.dropout2(x) + self.dropout1(y) + self.dropout2(self.act_func(self.linear_layer3(y) )  )

        x = self.act_func(self.linear_layer1(x))
        x = self.ln_1(x)

        if self.config.only_use_residual:
            y = x + self.act_func(self.linear_layer2(x) )
            z = x + y + self.act_func(self.linear_layer3(y) )
        elif self.config.only_use_residual_and_dropout:
            y = self.dropout1(x) + self.act_func(self.linear_layer2(x) )
            z = self.dropout2(x) + self.dropout1(y) + self.act_func(self.linear_layer3(y) )
        else:
            assert ValueError("config error")
        return z




class Ner_Model(nn.Module):
    def __init__(self,config, label_num, loss_name):
        super(Ner_Model, self).__init__()
        self.config = config
        # deberat model
        self.model = transformers.AutoModel.from_pretrained(config.model_name)

        hidden_size = config.hidden_size

        # using linear layer
        linear_layer = []
        if self.config.linear_layer_num  > 0:
            # using linear layer
            if self.config.only_use_standard_linear_layer:
                for out_dim in config.linear_layer[0:config.linear_layer_num]:
                    linear_layer.append( nn.Linear(in_features=hidden_size, out_features=out_dim) )
                    linear_layer.append( nn.ReLU() )
                    hidden_size = out_dim
                self.linear_model = nn.Sequential(*linear_layer)
            # just use dropout
            elif self.config.only_use_dropout:
                for i, out_dim in enumerate(config.linear_layer[0:config.linear_layer_num]):
                    linear_layer.append( nn.Linear(in_features=hidden_size, out_features=out_dim) )
                    linear_layer.append( nn.ReLU() )
                    linear_layer.append( nn.Dropout(config.dropout_prob[i]) )
                    hidden_size = out_dim
                self.linear_model = nn.Sequential(*linear_layer)
            else:
                # use 3 linear layer for skip 2 dropout
                assert config.linear_layer[0] == config.linear_layer[1] == config.linear_layer[2]
                assert len(config.dropout_prob) == 2
                self.linear_model = LinearResidualLayer(config, hidden_size,config.linear_layer[0])
                hidden_size = config.linear_layer[0]

        # whether to use lstm layer
        if config.lstm_layer_num > 0:
            self.lstm = LSTMEncoder(config,hidden_size)

        # identify label number
        self.label_num = label_num

        # whether to use bi-lstm layer
        if config.bi_lstm and config.lstm_layer_num > 0:
            hidden_size = hidden_size * 2

        self.classifier = nn.Linear(hidden_size, label_num)

        if loss_name == 'ce':
            self.loss_func = CE_Loss()
        elif loss_name == 'l1':
            self.loss_func = L1_Loss()
        elif loss_name == 'l2':
            self.loss_func = L2_Loss()
        elif loss_name == 'kl':
            self.loss_func = KLDivergenceLoss()
        elif loss_name == 'dlite':
            self.loss_func = DLITELoss()
        else:
            assert 1==0

        print("model configuration")
        print("%" * 20)
        print(self)
        print("%" * 20)

    def forward(self, input_ids, seq_length, attention_mask, labels):
        output = self.model(input_ids, attention_mask)
        sequence_output = output[0]
        if self.config.linear_layer_num > 0:
            sequence_output = self.linear_model(sequence_output)

        if self.config.lstm_layer_num > 0:
            sequence_output = self.lstm(sequence_output, seq_length)

        logit = self.classifier(sequence_output)
        loss = self.loss_func.loss(labels, logit, len(label2id))
        return loss, logit

In [ ]:
# Building optimizer
def get_optimizer(model, config):
    param_optimizer = list(model.named_parameters())

    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.1},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    optimizer = AdamW(optimizer_grouped_parameters,
                      betas=(0.9, 0.98),
                      lr=config.lr)
    return optimizer


## Defining the training function

In [ ]:

def evaluate(model, data_loader, mode="Validation"):
    ground_truth, predict = [], []
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples = 0
    eval_preds, eval_labels = [], []
    with torch.no_grad():
        for step, batch in enumerate(data_loader):
            attention_mask = batch["id"].ne(0)
            targets = batch['label_id']
            loss, logit = model(batch["id"], batch['seq_length'], attention_mask=attention_mask,
                                             labels=targets)
            eval_loss += loss.cpu().item()
            if (step+1) % 100==0:
                loss_step = eval_loss / (step+1)
                print(f"{mode} loss per 100 evaluation steps: {loss_step}")

            # compute training accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = logit.view(-1, len(label2id)) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            active_accuracy = flattened_targets.ne(-100) # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

            eval_preds.extend(predictions.tolist())
            eval_labels.extend(targets.tolist())

    eval_loss = eval_loss / (step+1)
    eval_accuracy = eval_accuracy / (step+1)
    eval_labels,eval_preds = [id2label[i] for i in eval_labels], [id2label[i] for i in eval_preds]


    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(eval_labels, eval_preds, average='micro')
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(eval_labels, eval_preds, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(eval_labels, eval_preds,average='weighted')

    p_r_f1 = [[round(precision_micro,4), round(recall_micro,4), round(f1_micro,4)],
              [round(precision_macro,4), round(recall_macro,4), round(f1_macro,4)],
              [round(precision_weighted,4), round(recall_weighted,4), round(f1_weighted,4)]]

    p_r_f1 = pd.DataFrame(p_r_f1, columns=['precision', 'recall', 'f1'], index=['micro', 'macro', 'weighted'])

    print(f"{mode} Loss: {eval_loss}")
    print(f"{mode} Accuracy: {eval_accuracy}")

    p_r_f1_each_label = classification_report(eval_labels, eval_preds)
    print(f"{mode} P-R-F1 for each label: \n{p_r_f1_each_label}")
    print(f"{mode} P-R-F1 tor all label: \n{p_r_f1}")
    print(f"{mode} steps: {(step+1)}")
    return eval_loss, p_r_f1, p_r_f1_each_label



## Running under 5 custom loss functions

In [ ]:
import warnings
warnings.filterwarnings('ignore')
loss_list = ['l1', 'l2', 'ce', 'kl', 'dlite']

In [ ]:
from sklearn.metrics import accuracy_score
def train(config,loss_name):
    print("=" * 100)
    print(f"loss_name: {loss_name}")
    model = Ner_Model(config, len(label2id), loss_name).to(config.device)
    optimizer = get_optimizer(model, config)

    valid_each_label_p_r_f1_list = []
    valid_p_r_f1_list = []
    test_each_label_p_r_f1_list = []
    test_p_r_f1_list = []

    valid_loss_list = []
    test_loss_list = []

    model.train()
    interval = 100
    for epoch in range(config.epochs):
        print(f"Training epoch: {epoch + 1}")
        tr_preds,tr_labels = [], []
        total_loss = 0.0
        tr_accuracy = 0.0
        # print("\n" + "-" * 30 + "\n")
        # print(f"epoch: {epoch},  train dataloader size: {len(train_dataloader)}")
        # print(f"epoch: {epoch},  valid dataloader size: {len(valid_dataloader)}")
        # print(f"epoch: {epoch},  test dataloader size: {len(test_dataloader)}")
        for step, batch in enumerate(train_dataloader):
            attention_mask = batch["id"].ne(0)
            targets = batch['label_id']
            loss, logit= model(batch["id"], batch['seq_length'], attention_mask=attention_mask,
                                             labels=targets)

            # compute training accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = logit.view(-1, len(label2id)) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            active_accuracy = flattened_targets.ne(-100) # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            tr_accuracy += tmp_tr_accuracy
            tr_preds.extend(predictions)
            tr_labels.extend(targets)

            total_loss += loss.item()
            if (step + 1) % interval == 0:
                print(f"Training loss per 100 training steps: {total_loss / (step+1)}")

            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

        print(f"Training loss epoch: {total_loss / (step+1)}")
        print(f"Training accuracy epoch: {tr_accuracy / (step+1)}")
        print(f"Training steps: {step+1}")
        print("\n\n")
        model.eval()


        valid_loss, valid_p_r_f1,  valid_each_label_p_r_f1 = evaluate(model,valid_dataloader, "Validation")
        valid_loss_list.append(valid_loss)
        valid_p_r_f1_list.append(valid_p_r_f1)
        valid_each_label_p_r_f1_list.append(valid_each_label_p_r_f1)

        print("\n\n")
        test_loss, test_p_r_f1,test_each_label_p_r_f1  = evaluate(model,test_dataloader, "Test")
        test_loss_list.append(test_loss)
        test_p_r_f1_list.append(test_p_r_f1)
        test_each_label_p_r_f1_list.append(test_each_label_p_r_f1)


        #print(f"epoch: {epoch}, train_loss: {train_loss}, \n{train_p_r_f1}")
        #print(f"epoch: {epoch}, valid_loss: {valid_loss}, \n{valid_p_r_f1}")
        #print(f"epoch: {epoch}, test_loss: {test_loss},  \n {test_p_r_f1}")
        model.train()
    return   {
              "valid_loss_list":valid_loss_list,
              "test_loss_list":test_loss_list,

              "valid_p_r_f1_list":valid_p_r_f1_list,
              "valid_each_label_p_r_f1_list":valid_each_label_p_r_f1_list,

              "test_p_r_f1_list":test_p_r_f1_list,
              "test_each_label_p_r_f1_list": test_each_label_p_r_f1_list}


result = {}
for loss_name in ['l1', 'l2', 'ce', 'kl', 'dlite']:
    r = train(Config, loss_name)
    result[loss_name] = r

loss_name: l1


pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

model configuration
%%%%%%%%%%%%%%%%%%%%
Ner_Model(
  (model): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
       

## Result Comparison after cleaning

In [ ]:
import pickle
with open("result.pkl", "wb") as f:
    pickle.dump(result, f)

#### Overall Result

In [ ]:
columns = ['loss', 'precision', 'recall', 'f1']
for t in ['micro', 'macro', 'weighted']:
    df = []
    for loss_name in loss_list:
        row = {'loss': loss_name}
        row['precision'] = result[loss_name]['test_p_r_f1_list'][-1].loc[t, 'precision']
        row['recall'] = result[loss_name]['test_p_r_f1_list'][-1].loc[t, 'recall']
        row['f1'] = result[loss_name]['test_p_r_f1_list'][-1].loc[t, 'f1']
        df.append(row)
    print("="*100)
    print(t)
    print(pd.DataFrame(df))

micro
    loss  precision  recall      f1
0     l1     0.9732  0.9732  0.9732
1     l2     0.9744  0.9744  0.9744
2     ce     0.9769  0.9769  0.9769
3     kl     0.9750  0.9750  0.9750
4  dlite     0.9733  0.9733  0.9733
macro
    loss  precision  recall      f1
0     l1     0.8824  0.9075  0.8937
1     l2     0.8834  0.9132  0.8973
2     ce     0.8961  0.9185  0.9070
3     kl     0.8987  0.9085  0.9029
4  dlite     0.8833  0.9121  0.8970
weighted
    loss  precision  recall      f1
0     l1     0.9749  0.9732  0.9739
1     l2     0.9759  0.9744  0.9749
2     ce     0.9778  0.9769  0.9773
3     kl     0.9763  0.9750  0.9754
4  dlite     0.9747  0.9733  0.9738


#### Each label Result

In [ ]:

print("test dataset")
for loss_name in loss_list:
    print("-"*50)
    print(loss_name)
    print(result[loss_name]['test_each_label_p_r_f1_list'][-1])


test dataset
--------------------------------------------------
l1
              precision    recall  f1-score   support

       B-LOC       0.93      0.94      0.93      1668
      B-MISC       0.78      0.85      0.81       702
       B-ORG       0.89      0.91      0.90      1661
       B-PER       0.98      0.94      0.96      1617
       I-LOC       0.89      0.91      0.90      1394
      I-MISC       0.61      0.75      0.67       736
       I-ORG       0.89      0.93      0.91      2804
       I-PER       0.98      0.95      0.97      3810
           O       0.99      0.99      0.99     47094

    accuracy                           0.97     61486
   macro avg       0.88      0.91      0.89     61486
weighted avg       0.97      0.97      0.97     61486

--------------------------------------------------
l2
              precision    recall  f1-score   support

       B-LOC       0.91      0.95      0.93      1668
      B-MISC       0.80      0.86      0.83       702
       B-OR